In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.2 MB/s eta 0:00:00


In [ ]:
!unzip /content/source_vs_binary.zip
#!unzip /content/zlib.zip

Streaming output truncated to the last 5000 lines.
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function0/diff_OpenSSL_1_1_1i_vs_OpenSSL_1_1_1j@binary_new.txt  
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function0/diff_OpenSSL_1_1_1i_vs_OpenSSL_1_1_1j@binary_old.txt  
   creating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function1/
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function1/diff_openssl-3.0.0-alpha8_vs_openssl-3.0.0-alpha9.txt  
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function1/diff_openssl-3.0.0-alpha8_vs_openssl-3.0.0-alpha9@binary_new.txt  
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function1/diff_openssl-3.0.0-alpha8_vs_openssl-3.0.0-alpha9@binary_old.txt  
  inflating: oldestVersion_vs_newestVersion/openssl/ssl_record/rec_layer_d1.c/function1/diff_OpenSSL_1_1_0-pre3_vs_OpenS

In [ ]:
cd /content/drive/MyDrive/Colab/Research/

/content/drive/MyDrive/Colab/Research


In [ ]:
import os
import torch
import torch.nn.functional as F
import binaryBERT
import codeBERT
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from config import *
from transformers import set_seed, AutoTokenizer
from collections import defaultdict
from tqdm import tqdm
import config
import random
import numpy as np
import pickle
import math
import copy
from sklearn.metrics import precision_score, recall_score, f1_score
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA'] = '1'
seed = 628
set_seed(seed)

dataset_path = "/content/source_vs_binary/"
model_path = "/content/drive/MyDrive/Colab/Research/model_state"
sourceCode_Model = codeBERT.MyModel(codeBERT.pretrain_model,codeBERT.config).to(device)
binaryCode_Model = binaryBERT.MyModel().to(device)
binaryCode_Model.load_state_dict(torch.load('model_state/model.dat'), strict=False)
sourceCode_Model.load_state_dict(torch.load('model_state/codeBert_finetune.dat'), strict=False)

tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")
binaryDict = binaryBERT.load_tokens()
func_length = 512

Some weights of the model checkpoint at neulab/codebert-c were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-c and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The argument `trust_re

In [ ]:
class WarmupLinearSchedule(LambdaLR):
    """ Linear warmup and then linear decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Linearly decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps.
    """
    def __init__(self, optimizer, warmup_steps, t_total, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        super(WarmupLinearSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        return max(0.0, float(self.t_total - step) / float(max(1.0, self.t_total - self.warmup_steps)))


class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(d_model * 2, d_model * 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(d_model * 2, 3)
        self.sigmoid = nn.Sigmoid()

    def forward(self,source_output, binary_output):
        x = torch.cat([source_output,binary_output], dim=-1)
        x = F.gelu(self.fc1(x))
        x = self.fc2(x)
        return x
        #return self.sigmoid(x)

# class MLP(nn.Module):
#     def __init__(self):
#         super(MLP,self).__init__()
#         self.fc1 = nn.Linear(d_model * 3, d_model * 3)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.1)
#         self.fc2 = nn.Linear(d_model * 3, 3)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self,new, old, binary_output):
#         x = torch.cat([new, old, binary_output], dim=-1)
#         x = F.gelu(self.fc1(x))
#         x = self.fc2(x)
#         return x
#         #return self.sigmoid(x)

In [ ]:
def getAllPath():
    path_list = []
    for dirs in tqdm(os.walk(dataset_path)):
        if len(dirs[-1]) > 0:
            d = {}
            for f in dirs[-1]:
                if not 'binary_old' in f and not 'binary_new' in f:
                  d[f'{dirs[0]}{os.sep}{f}'] = []

            for f in dirs[-1]:
              if 'binary_old' in f or 'binary_new' in f:
                for k in d:
                  if f.split("@")[0] + ".txt" == k.split(os.sep)[-1]:
                    d[k].append(f'{dirs[0]}{os.sep}{f}')
            for k in d:
              filelist = []
              filelist.append(k)
              filelist.extend(d[k])
              if len(filelist) > 1:
                path_list.append(filelist)

    return path_list


def binary_tokenize(file):
    cls_id = binaryDict['<cls>']
    unk_id = binaryDict['<unk>']

    token = []
    token_id_list = [cls_id]
    with open(file,'r') as file:
        for line in file:
            line = line.lstrip().rstrip().split(" ")
            line = [int(i) for i in line]
            token.extend(line)
    # for t in token[:func_length-1]:
    #     token_id_list.append(binaryDict.get(t, unk_id))

    return token

def yield_functions(shuffle=False, seed=None, split_ratio=None, mode=None, finetune=False):
    path_list = getAllPath()
    total = len(path_list)
    r = random.Random(seed)

    if split_ratio is not None:
        split_point = int(len(path_list) * split_ratio)
        if mode == 'train':
            path_list = path_list[:split_point]
        elif mode == 'test':
            path_list = path_list[split_point:]

    if shuffle:
      r.shuffle(path_list)

    maxSequenceLength = 512
    count = 0
    for file_list in tqdm(path_list):
        diff_fp = ''
        binary_old_fp = ''
        binary_new_fp = ''
        for fp in file_list:
            if 'binary_old' not in fp and 'binary_new' not in fp:
                diff_fp = fp
            elif 'binary_old' in fp:
                binary_old_fp = fp
            elif 'binary_new' in fp:
                binary_new_fp = fp

        token_list = {}
        lno = 1
        func_length = 0
        func_name_new = None
        func_name_old = None
        try:
            with open(diff_fp,"r") as file:
                codeLines = file.readlines()
                for i in range(len(codeLines)):
                    # empty line
                    line = codeLines[i]
                    if i == 0:
                      line = line.lstrip().rstrip().split("#@#")
                      func_name_new = line[0].lstrip().rstrip()
                      func_name_old = line[1].lstrip().rstrip()
                      continue
                    if '@@##$$##@@' in line:
                      line = line.split("@@##$$##@@")[-1]
                    if len(line.lstrip().strip().rstrip()) == 0:
                        continue

                    #line = line.lstrip().rstrip()
                    line = line.rstrip()

                    if line.startswith("+") or line.startswith("-"):
                        # remove space before the code starts
                        #line = line[0] + " " + line[1:].lstrip()
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        func_length += len(token_list[lno])
                    else:
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        func_length += len(token_list[lno])
                    lno += 1
            file.close()

            if func_length > 511:
                count += 1
                continue

            d = [12,2744] # represent - and +
            new = [0] + [token for line in sorted(token_list.keys()) if token_list[line][0] == 2744 or token_list[line][0] not in d for token in token_list[line]]
            old = [0] + [token for line in sorted(token_list.keys()) if token_list[line][0] == 12 or token_list[line][0] not in d for token in token_list[line]]
            diff = [0] + [token for line in sorted(token_list.keys()) for token in token_list[line]]

            binary_new = 'None'
            binary_old = 'None'
            if binary_new_fp != '':
                binary_new = binary_tokenize(binary_new_fp)
            if binary_old_fp != '':
                binary_old = binary_tokenize(binary_old_fp)

            if len(binary_new) > 512:
              binary_new = 'None'
            if len(binary_old) > 512:
              binary_old = 'None'
            if binary_new == 'None' and binary_old == 'None':
              continue

            if binary_new == binary_old:
              yield diff, new, old, binary_new, 'None', diff_fp
            else:
              yield diff, new, old, binary_new, binary_old, diff_fp

        except Exception as e:
            print(e)
            print(fp)


def data_loader(binaryDict, batch_size, func_len, mode):
    b_pad_id = binaryDict['<pad>']
    s_pad_id = tokenizer.pad_token_id

    output_d = {}
    allFunctions = {}
    total_functions = 0
    r = random.Random(seed)
    count_new = 0
    count_old = 0
    for diff, new, old, binary_new, binary_old, diff_fp in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,finetune=True):
        output_d[diff_fp] = [[diff,new,old], binary_new, binary_old]
        if binary_new != 'None':
            allFunctions[f'{diff_fp}@binary_new'] = binary_new
            count_new += 1
        if binary_old != 'None':
            allFunctions[f'{diff_fp}@binary_old'] = binary_old
            count_old += 1
        total_functions += 1

    print(f'new: {count_new}')
    print(f'old: {count_old}')
    print('total_functions:', total_functions)
    diff_output = []
    binary_output = []
    padding_list = []
    attention_mask = []
    functionNames = list(allFunctions.keys())
    label = []
    cl = {'patched':0,'unpatched':1,'unrelated':2}
    total = 0
    count = batch_size
    for key in tqdm(output_d):
        while True:
            unrelated_example = r.choice(functionNames)
            fname_old = key + '@binary_old'
            fname_new = key + '@binary_new'
            if unrelated_example != fname_old and unrelated_example != fname_new:
                break
        funcList = []
        if output_d[key][1] != 'None':
            #funcList.append(['patched',output_d[key][1],output_d[key][0][1]])
            funcList.append(['patched',output_d[key][1],output_d[key][0][0]])
        if output_d[key][2] != 'None':
            #funcList.append(['unpatched',output_d[key][2],output_d[key][0][2]])
            funcList.append(['unpatched',output_d[key][2],output_d[key][0][0]])

        funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][0]])
        #funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][1]])
        #funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][2]])


        for i in range(0,len(funcList)):
            #diff_token_id_list = list(output_d[key][0])
            diff_token_id_list = copy.deepcopy(funcList[i][-1])
            binary_token_id_list = funcList[i][1]
            class_label = cl[funcList[i][0]]

            attention_mask.append([1] * len(diff_token_id_list))
            if (l := len(diff_token_id_list)) < func_len:
                diff_token_id_list += [s_pad_id] * (func_len - l)
                attention_mask[-1] += [0] * (func_len - l)


            padding_list.append([False] * len(binary_token_id_list))
            if (l := len(binary_token_id_list)) < func_len:
                binary_token_id_list += [b_pad_id] * (func_len - l)
                padding_list[-1] += [True] * (func_len - l)


            diff_output.append(diff_token_id_list)
            binary_output.append(binary_token_id_list)
            l = [0] * 3
            l[class_label] = 1
            label.append(l)

        count -= 1
        if count == 0 or total == len(output_d) - 1:
            total += 1
            out_data_diff = np.asarray(diff_output, dtype=np.int64)
            out_data_binary = np.asarray(binary_output,dtype=np.int64)
            out_attention_mask = np.asarray(attention_mask,dtype=np.int64)
            out_padding_list = np.asarray(padding_list,dtype=bool)
            out_label = np.asarray(label,dtype=float)

            diff_output = []
            binary_output = []
            padding_list = []
            attention_mask = []
            label = []
            count = batch_size

            yield out_data_diff, out_data_binary, out_attention_mask,out_padding_list, out_label


def test_data_loader(binaryDict, batch_size, func_len, mode):
    b_pad_id = binaryDict['<pad>']
    s_pad_id = tokenizer.pad_token_id

    output_d = {}
    allFunctions = {}
    total_functions = 0
    r = random.Random(seed)
    count_new = 0
    count_old = 0
    for diff, new, old, binary_new, binary_old, diff_fp in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,finetune=True):
        output_d[diff_fp] = [[diff,new,old], binary_new, binary_old]
        if binary_new != 'None':
            allFunctions[f'{diff_fp}@binary_new'] = binary_new
            count_new += 1
        if binary_old != 'None':
            allFunctions[f'{diff_fp}@binary_old'] = binary_old
            count_old += 1
        total_functions += 1

    print(f'new: {count_new}')
    print(f'old: {count_old}')
    print('total_functions:', total_functions)
    diff_output = []
    binary_output = []
    padding_list = []
    attention_mask = []
    functionNames = list(allFunctions.keys())
    label = []
    cl = {'patched':0,'unpatched':1,'unrelated':2}
    total = 0
    count = batch_size
    for key in tqdm(output_d):
        while True:
            unrelated_example = r.choice(functionNames)
            fname_old = key + '@binary_old'
            fname_new = key + '@binary_new'
            if unrelated_example != fname_old and unrelated_example != fname_new:
                break
        funcList = []
        if output_d[key][1] != 'None':
            funcList.append(['patched',output_d[key][1],output_d[key][0][1]])
            if output_d[key][2] != 'None':
              funcList.append(['unrelated',output_d[key][2],output_d[key][0][1]])
            #funcList.append(['patched',output_d[key][1],output_d[key][0][0]])
        if output_d[key][2] != 'None':
            funcList.append(['unpatched',output_d[key][2],output_d[key][0][2]])
            if output_d[key][1] != 'None':
              funcList.append(['unrelated',output_d[key][1],output_d[key][0][2]])
            #funcList.append(['unpatched',output_d[key][2],output_d[key][0][0]])
        funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][1]])
        #funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][0]])
        funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][2]])

        for i in range(0,len(funcList)):
            #diff_token_id_list = list(output_d[key][0])
            diff_token_id_list = copy.deepcopy(funcList[i][-1])
            binary_token_id_list = copy.deepcopy(funcList[i][1])
            class_label = cl[funcList[i][0]]

            attention_mask.append([1] * len(diff_token_id_list))
            if (l := len(diff_token_id_list)) < func_len:
                diff_token_id_list += [s_pad_id] * (func_len - l)
                attention_mask[-1] += [0] * (func_len - l)


            padding_list.append([False] * len(binary_token_id_list))
            if (l := len(binary_token_id_list)) < func_len:
                binary_token_id_list += [b_pad_id] * (func_len - l)
                padding_list[-1] += [True] * (func_len - l)


            diff_output.append(diff_token_id_list)
            binary_output.append(binary_token_id_list)
            l = [0] * 3
            l[class_label] = 1
            label.append(l)

        count -= 1
        if count == 0 or total == len(output_d) - 1:
            total += 1
            out_data_diff = np.asarray(diff_output, dtype=np.int64)
            out_data_binary = np.asarray(binary_output,dtype=np.int64)
            out_attention_mask = np.asarray(attention_mask,dtype=np.int64)
            out_padding_list = np.asarray(padding_list,dtype=bool)
            out_label = np.asarray(label,dtype=float)

            diff_output = []
            binary_output = []
            padding_list = []
            attention_mask = []
            label = []
            count = batch_size

            yield out_data_diff, out_data_binary, out_attention_mask,out_padding_list, out_label


def test_data_loader1(binaryDict, batch_size, func_len, mode):
    b_pad_id = binaryDict['<pad>']
    s_pad_id = tokenizer.pad_token_id

    output_d = {}
    allFunctions = {}
    total_functions = 0
    r = random.Random(seed)
    count_new = 0
    count_old = 0
    for diff, new, old, binary_new, binary_old, diff_fp in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,finetune=True):
        output_d[diff_fp] = [[diff,new,old], binary_new, binary_old]
        if binary_new != 'None':
            allFunctions[f'{diff_fp}@binary_new'] = binary_new
            count_new += 1
        if binary_old != 'None':
            allFunctions[f'{diff_fp}@binary_old'] = binary_old
            count_old += 1
        total_functions += 1

    print(f'new: {count_new}')
    print(f'old: {count_old}')
    print('total_functions:', total_functions)
    new_output = []
    old_output = []

    binary_output = []
    padding_list = []
    new_attention_mask = []
    old_attention_mask = []
    functionNames = list(allFunctions.keys())
    label = []
    cl = {'patched':0,'unpatched':1,'unrelated':2}
    total = 0
    count = batch_size
    for key in tqdm(output_d):
        while True:
            unrelated_example = r.choice(functionNames)
            fname_old = key + '@binary_old'
            fname_new = key + '@binary_new'
            if unrelated_example != fname_old and unrelated_example != fname_new:
                break
        funcList = []
        if output_d[key][1] != 'None':
            funcList.append(['patched',output_d[key][1],output_d[key][0][1],output_d[key][0][2]])

        if output_d[key][2] != 'None':
            funcList.append(['unpatched',output_d[key][2],output_d[key][0][1],output_d[key][0][2]])

        funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][1],output_d[key][0][2]])

        for i in range(0,len(funcList)):
            #diff_token_id_list = list(output_d[key][0])
            new_token_id_list = copy.deepcopy(funcList[i][-2])
            old_token_id_list = copy.deepcopy(funcList[i][-1])
            binary_token_id_list = copy.deepcopy(funcList[i][1])
            class_label = cl[funcList[i][0]]

            new_attention_mask.append([1] * len(new_token_id_list))
            if (l := len(new_token_id_list)) < func_len:
                new_token_id_list += [s_pad_id] * (func_len - l)
                new_attention_mask[-1] += [0] * (func_len - l)

            old_attention_mask.append([1] * len(old_token_id_list))
            if (l := len(old_token_id_list)) < func_len:
                old_token_id_list += [s_pad_id] * (func_len - l)
                old_attention_mask[-1] += [0] * (func_len - l)

            padding_list.append([False] * len(binary_token_id_list))
            if (l := len(binary_token_id_list)) < func_len:
                binary_token_id_list += [b_pad_id] * (func_len - l)
                padding_list[-1] += [True] * (func_len - l)


            new_output.append(new_token_id_list)
            old_output.append(old_token_id_list)
            binary_output.append(binary_token_id_list)
            l = [0] * 3
            l[class_label] = 1
            label.append(l)

        count -= 1
        if count == 0 or total == len(output_d) - 1:
            total += 1
            diff_new_output = np.asarray(new_output, dtype=np.int64)
            diff_old_output = np.asarray(old_output, dtype=np.int64)

            out_data_binary = np.asarray(binary_output,dtype=np.int64)
            out_new_attention_mask = np.asarray(new_attention_mask,dtype=np.int64)
            out_old_attention_mask = np.asarray(old_attention_mask,dtype=np.int64)
            out_padding_list = np.asarray(padding_list,dtype=bool)
            out_label = np.asarray(label,dtype=float)

            new_output = []
            old_output = []
            binary_output = []
            padding_list = []
            new_attention_mask = []
            old_attention_mask = []
            label = []
            count = batch_size

            yield diff_new_output, diff_old_output, out_data_binary, out_new_attention_mask, out_old_attention_mask, out_padding_list, out_label

def test_data_loader2(binaryDict, batch_size, func_len, mode):
    b_pad_id = binaryDict['<pad>']
    s_pad_id = tokenizer.pad_token_id

    output_d = {}
    allFunctions = {}
    total_functions = 0
    r = random.Random(seed)
    count_new = 0
    count_old = 0
    for diff, new, old, binary_new, binary_old, diff_fp in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,finetune=True):
        output_d[diff_fp] = [[diff,new,old], binary_new, binary_old]
        if binary_new != 'None':
            allFunctions[f'{diff_fp}@binary_new'] = binary_new
            count_new += 1
        if binary_old != 'None':
            allFunctions[f'{diff_fp}@binary_old'] = binary_old
            count_old += 1
        total_functions += 1

    print(f'new: {count_new}')
    print(f'old: {count_old}')
    print('total_functions:', total_functions)
    new_output = []
    old_output = []

    binary_output = []
    padding_list = []
    new_attention_mask = []
    old_attention_mask = []
    functionNames = list(allFunctions.keys())
    label = []
    cl = {'patched':0,'unpatched':1,'unrelated':2}
    total = 0
    count = batch_size
    for key in tqdm(output_d):
        while True:
            unrelated_example = r.choice(functionNames)
            fname_old = key + '@binary_old'
            fname_new = key + '@binary_new'
            if unrelated_example != fname_old and unrelated_example != fname_new:
                break
        funcList = []
        if output_d[key][1] != 'None':
            funcList.append(['patched',output_d[key][1],output_d[key][0][1],output_d[key][0][2]])
            # if output_d[key][2] != 'None':
            #   funcList.append(['unrelated',output_d[key][2],output_d[key][0][1],output_d[key][0][2]])

        if output_d[key][2] != 'None':
            funcList.append(['unpatched',output_d[key][2],output_d[key][0][1],output_d[key][0][2]])
            # if output_d[key][1] != 'None':
            #   funcList.append(['unrelated',output_d[key][1],output_d[key][0][1],output_d[key][0][2]])

        funcList.append(['unrelated',allFunctions[unrelated_example],output_d[key][0][1],output_d[key][0][2]])

        for i in range(0,len(funcList)):
            #diff_token_id_list = list(output_d[key][0])
            diff_token_id_list = copy.deepcopy(funcList[i][2:])
            binary_token_id_list = copy.deepcopy(funcList[i][1])
            class_label = cl[funcList[i][0]]

            new_attention_mask.append([1] * len(diff_token_id_list[0]))
            if (l := len(diff_token_id_list[0])) < func_len:
                diff_token_id_list[0] += [s_pad_id] * (func_len - l)
                new_attention_mask[-1] += [0] * (func_len - l)

            old_attention_mask.append([1] * len(diff_token_id_list[1]))
            if (l := len(diff_token_id_list[1])) < func_len:
                diff_token_id_list[1] += [s_pad_id] * (func_len - l)
                old_attention_mask[-1] += [0] * (func_len - l)

            padding_list.append([False] * len(binary_token_id_list))
            if (l := len(binary_token_id_list)) < func_len:
                binary_token_id_list += [b_pad_id] * (func_len - l)
                padding_list[-1] += [True] * (func_len - l)


            new_output.append(diff_token_id_list[0])
            old_output.append(diff_token_id_list[1])
            binary_output.append(binary_token_id_list)
            l = [0] * 3
            l[class_label] = 1
            label.append(l)

        count -= 1
        if count == 0 or total == len(output_d) - 1:
            total += 1
            diff_new_output = np.asarray(new_output, dtype=np.int64)
            diff_old_output = np.asarray(old_output, dtype=np.int64)
            out_data_binary = np.asarray(binary_output,dtype=np.int64)
            out_new_attention_mask = np.asarray(new_attention_mask,dtype=np.int64)
            out_old_attention_mask = np.asarray(old_attention_mask,dtype=np.int64)
            out_padding_list = np.asarray(padding_list,dtype=bool)
            out_label = np.asarray(label,dtype=float)

            new_output = []
            old_output = []
            binary_output = []
            padding_list = []
            new_attention_mask = []
            old_attention_mask = []
            label = []
            count = batch_size

            yield diff_new_output, diff_old_output, out_data_binary, out_new_attention_mask, out_old_attention_mask, out_padding_list, out_label

def sourceCodeInfer(data, attention_mask):
  # sourceCode_Model.eval()
  #with torch.no_grad():
  return sourceCode_Model(data,attention_mask)

def binaryCodeInfer(data, padding):
  #with torch.no_grad():
  return binaryCode_Model(data,padding,ft=True)

In [ ]:
def train():
    torch.cuda.empty_cache()
    lr = 1e-5
    MLP_Model = MLP().to(device)
    total_epoch = 5
    batch_size = 2
    optimizer = torch.optim.AdamW([{'params': MLP_Model.parameters()}, {'params': binaryCode_Model.parameters()}, {'params': sourceCode_Model.parameters()}], lr=lr)
    #optimizer = torch.optim.AdamW([{'params': MLP_Model.parameters()}, {'params': binaryCode_Model.parameters()}], lr=lr)
    total_instance = 8104
    #total_instance = 5692
    total_train_step = total_instance / batch_size
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=total_epoch * total_train_step * 0.1, t_total=total_epoch * total_train_step)
    cur_step = 0

    for epoch in range(total_epoch):
      global_loss = 0.0
      for data_diff, data_binary, attention_mask,padding_list, label in test_data_loader(binaryDict,batch_size,512,'train'):
        data_diff = torch.as_tensor(data_diff, device=device)
        data_binary = torch.as_tensor(data_binary, device=device).t()
        attention_mask = torch.as_tensor(attention_mask, device=device)
        label = torch.as_tensor(label, device=device)
        padding_list = torch.as_tensor(padding_list,device=device)

        source_output = sourceCodeInfer(data_diff, attention_mask)
        binary_output = binaryCodeInfer(data_binary, padding_list)

        output = MLP_Model(source_output,binary_output)

        loss = F.cross_entropy(output, label)
        global_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(MLP_Model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(binaryCode_Model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(sourceCode_Model.parameters(), 1.0)
        optimizer.step()
        # reset the gradients to zero
        optimizer.zero_grad()
        # update the learning rate
        scheduler.step()

        cur_step += 1
        if cur_step % 10 == 0:
          print(f'{epoch=}, {cur_step=}, loss={loss.item()}')
        if cur_step % 500 == 0:
          if os.path.exists(model_path):
              torch.save(MLP_Model.state_dict(), 'model_state/classification_model1.dat')
              torch.save(sourceCode_Model.state_dict(), 'model_state/sourceCode_Model1.dat')
              torch.save(binaryCode_Model.state_dict(), 'model_state/binaryCode_Model1.dat')
      print(f'average loss: {global_loss / total_train_step}')
      #test(MLP_Model)

# def train():
#     torch.cuda.empty_cache()
#     lr = 1e-5
#     MLP_Model = MLP().to(device)
#     total_epoch = 5
#     batch_size = 5
#     optimizer = torch.optim.AdamW([{'params': MLP_Model.parameters()}, {'params': binaryCode_Model.parameters()}, {'params': sourceCode_Model.parameters()}], lr=lr)
#     #optimizer = torch.optim.AdamW([{'params': MLP_Model.parameters()}, {'params': binaryCode_Model.parameters()}], lr=lr)
#     total_instance = 12889
#     total_train_step = total_instance / batch_size
#     scheduler = WarmupLinearSchedule(optimizer, warmup_steps=total_epoch * total_train_step * 0.1, t_total=total_epoch * total_train_step)
#     cur_step = 0
#     for epoch in range(total_epoch):
#       global_loss = 0.0
#       for data_diff, data_old, data_binary, diff_attention_mask, old_attention_mask,padding_list, label in test_data_loader2(binaryDict, batch_size,512, 'train'):
#         data_diff = torch.as_tensor(data_diff, device=device)
#         data_old = torch.as_tensor(data_old, device=device)

#         data_binary = torch.as_tensor(data_binary, device=device).t()
#         old_attention_mask = torch.as_tensor(old_attention_mask, device=device)
#         diff_attention_mask = torch.as_tensor(diff_attention_mask, device=device)

#         label = torch.as_tensor(label, device=device)
#         padding_list = torch.as_tensor(padding_list,device=device)

#         source_output_diff = sourceCodeInfer(data_diff, diff_attention_mask)
#         source_output_old = sourceCodeInfer(data_old, old_attention_mask)

#         binary_output = binaryCodeInfer(data_binary, padding_list)

#         output = MLP_Model(source_output_diff,source_output_old,binary_output)

#         loss = F.cross_entropy(output, label)
#         global_loss += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(MLP_Model.parameters(), 1.0)
#         torch.nn.utils.clip_grad_norm_(binaryCode_Model.parameters(), 1.0)
#         torch.nn.utils.clip_grad_norm_(sourceCode_Model.parameters(), 1.0)
#         optimizer.step()
#         # reset the gradients to zero
#         optimizer.zero_grad()
#         # update the learning rate
#         scheduler.step()

#         cur_step += 1
#         if cur_step % 10 == 0:
#           print(f'{epoch=}, {cur_step=}, loss={loss.item()}')
#         if cur_step % 500 == 0:
#           if os.path.exists(model_path):
#               torch.save(MLP_Model.state_dict(), 'model_state/classification_model2.dat')
#               torch.save(sourceCode_Model.state_dict(), 'model_state/sourceCode_Model2.dat')
#               torch.save(binaryCode_Model.state_dict(), 'model_state/binaryCode_Model2.dat')
#       print(f'average loss: {global_loss / total_train_step}')
#       test1(MLP_Model)

def accumulate_results(y_true_accum, y_pred_accum, y_true_batch, y_pred_batch):
    """
    Accumulate the predictions and true labels over multiple batches.
    Args:
        y_true_accum: accumulated true labels, shape=(num_samples, num_labels)
        y_pred_accum: accumulated predicted labels, shape=(num_samples, num_labels)
        y_true_batch: true labels for the current batch, shape=(batch_size, num_labels)
        y_pred_batch: predicted labels for the current batch, shape=(batch_size, num_labels)
    Returns:
        Updated accumulated true labels and predicted labels.
    """
    if y_true_accum is None:
        y_true_accum = y_true_batch
        y_pred_accum = y_pred_batch
    else:
        y_true_accum = np.concatenate([y_true_accum, y_true_batch], axis=0)
        y_pred_accum = np.concatenate([y_pred_accum, y_pred_batch], axis=0)
    return y_true_accum, y_pred_accum

def test(model):
  #model.load_state_dict(torch.load('model_state/classification_model2.dat'), strict=False)
  model.eval()
  binaryCode_Model.eval()
  sourceCode_Model.eval()
  y_true_res = None
  y_pred_res = None
  with torch.no_grad():
    for data_diff, data_binary, attention_mask,padding_list, label in test_data_loader(binaryDict, 2,512, 'test'):
      data_diff = torch.as_tensor(data_diff, device=device)
      data_binary = torch.as_tensor(data_binary, device=device).t()
      attention_mask = torch.as_tensor(attention_mask, device=device)
      label = torch.as_tensor(label, device=device)
      padding_list = torch.as_tensor(padding_list,device=device)

      source_output = sourceCodeInfer(data_diff, attention_mask)
      binary_output = binaryCodeInfer(data_binary, padding_list)

      output = model(source_output,binary_output)
      # print()
      # print(output)
      # print(output.argmax(dim=1).unsqueeze(1))
      one_hot = torch.zeros((output.size(0), 3),device=device)
      one_hot.scatter_(1, output.argmax(dim=1).unsqueeze(1), 1)
      # print(one_hot)
      # print(label)
      # input()
      y_true_res, y_pred_res = accumulate_results(y_true_res,y_pred_res,label.cpu().numpy(),one_hot.cpu().numpy())

    recall = recall_score(y_true_res, y_pred_res, average=None)
    precision = precision_score(y_true_res, y_pred_res, average=None)
    f1 = f1_score(y_true_res, y_pred_res, average=None)

    # Calculate macro and weighted average scores
    macro_recall = np.mean(recall)
    macro_precision = np.mean(precision)
    macro_f1 = np.mean(f1)
    weighted_recall = np.average(recall, weights=np.sum(y_true_res, axis=0))
    weighted_precision = np.average(precision, weights=np.sum(y_true_res, axis=0))
    weighted_f1 = np.average(f1, weights=np.sum(y_true_res, axis=0))

    # Create a dictionary with the results
    results = {"recall": recall, "precision": precision, "f1": f1,
               "macro_recall": macro_recall, "macro_precision": macro_precision, "macro_f1": macro_f1,
               "weighted_recall": weighted_recall, "weighted_precision": weighted_precision, "weighted_f1": weighted_f1}
    print()
    print(f'macro_recall: {results["macro_recall"]}  macro_precision: {results["macro_precision"]} macro_F1: {results["macro_f1"]}')
    print(f'weighted_recall: {results["weighted_recall"]}  weighted_precision: {results["weighted_precision"]} weighted_F1: {results["weighted_f1"]}')
    model.train()
    binaryCode_Model.train()
    sourceCode_Model.train()

def test1(model):
  # model = MLP().to(device)
  #model.load_state_dict(torch.load('model_state/classification_model2.dat'), strict=False)
  model.eval()
  binaryCode_Model.eval()
  sourceCode_Model.eval()
  y_true_res = None
  y_pred_res = None
  with torch.no_grad():
    for data_new, data_old, data_binary, new_attention_mask, old_attention_mask,padding_list, label in test_data_loader2(binaryDict, 3,512, 'test'):
        data_new = torch.as_tensor(data_new, device=device)
        data_old = torch.as_tensor(data_old, device=device)
        data_binary = torch.as_tensor(data_binary, device=device).t()
        new_attention_mask = torch.as_tensor(new_attention_mask, device=device)
        old_attention_mask = torch.as_tensor(old_attention_mask, device=device)
        label = torch.as_tensor(label, device=device)
        padding_list = torch.as_tensor(padding_list,device=device)

        source_output_new = sourceCodeInfer(data_new, new_attention_mask)
        source_output_old = sourceCodeInfer(data_old, old_attention_mask)
        binary_output = binaryCodeInfer(data_binary, padding_list)

        output = model(source_output_new,source_output_old,binary_output)

        one_hot = torch.zeros((output.size(0), 3),device=device)
        one_hot.scatter_(1, output.argmax(dim=1).unsqueeze(1), 1)

        y_true_res, y_pred_res = accumulate_results(y_true_res,y_pred_res,label.cpu().numpy(),one_hot.cpu().numpy())

    recall = recall_score(y_true_res, y_pred_res, average=None)
    precision = precision_score(y_true_res, y_pred_res, average=None)
    f1 = f1_score(y_true_res, y_pred_res, average=None)

    # Calculate macro and weighted average scores
    macro_recall = np.mean(recall)
    macro_precision = np.mean(precision)
    macro_f1 = np.mean(f1)
    weighted_recall = np.average(recall, weights=np.sum(y_true_res, axis=0))
    weighted_precision = np.average(precision, weights=np.sum(y_true_res, axis=0))
    weighted_f1 = np.average(f1, weights=np.sum(y_true_res, axis=0))

    # Create a dictionary with the results
    results = {"recall": recall, "precision": precision, "f1": f1,
                "macro_recall": macro_recall, "macro_precision": macro_precision, "macro_f1": macro_f1,
                "weighted_recall": weighted_recall, "weighted_precision": weighted_precision, "weighted_f1": weighted_f1}
    print()
    print(f'macro_recall: {results["macro_recall"]}  macro_precision: {results["macro_precision"]} macro_F1: {results["macro_f1"]}')
    print(f'weighted_recall: {results["weighted_recall"]}  weighted_precision: {results["weighted_precision"]} weighted_F1: {results["weighted_f1"]}')

    model.train()
    binaryCode_Model.train()
    sourceCode_Model.train()

def test2(model):
  # model = MLP().to(device)
  #model.load_state_dict(torch.load('model_state/classification_model2.dat'), strict=False)
  model.eval()
  binaryCode_Model.eval()
  sourceCode_Model.eval()
  y_true_res = None
  y_pred_res = None
  with torch.no_grad():
    for data_new, data_old, data_binary, new_attention_mask, old_attention_mask,padding_list, label in test_data_loader1(binaryDict, 1,512, 'test'):
        data_new = torch.as_tensor(data_new, device=device)
        data_old = torch.as_tensor(data_old, device=device)
        data_binary = torch.as_tensor(data_binary, device=device).t()
        new_attention_mask = torch.as_tensor(new_attention_mask, device=device)
        old_attention_mask = torch.as_tensor(old_attention_mask, device=device)
        label = torch.as_tensor(label, device=device)
        padding_list = torch.as_tensor(padding_list,device=device)

        source_output_new = sourceCodeInfer(data_new, new_attention_mask)
        source_output_old = sourceCodeInfer(data_old, old_attention_mask)
        binary_output = binaryCodeInfer(data_binary, padding_list)

        new_output = model(source_output_new,binary_output)
        old_output = model(source_output_old,binary_output)

        new_val, new_index = torch.max(new_output,dim=1)
        old_val, old_index = torch.max(old_output,dim=1)
        # print()
        # print(f'new: {new_output.argmax(dim=1)}')
        # print(f'new_val: {new_output}')
        # print(f'old: {old_output.argmax(dim=1)}')
        # print(f'old_val: {old_output}')
        final_output = torch.zeros_like(new_index)
        for i in range(new_output.size(0)):
          if new_index[i] == old_index[i] or new_index[i] != 2 and old_index[i] == 2:
            final_output[i] = new_index[i]
          elif new_index[i] == 2 and old_index[i] != 2:
            final_output[i] = old_index[i]
          elif new_val[i] > old_val[i]:
            final_output[i] = new_index[i]
          else:
            final_output[i] = old_index[i]

        #print(final_output.unsqueeze(1))
        one_hot = torch.zeros((new_output.size(0), 3),device=device)
        one_hot.scatter_(1, final_output.unsqueeze(1), 1)
        # print(one_hot)
        # print(label)
        # input()

        y_true_res, y_pred_res = accumulate_results(y_true_res,y_pred_res,label.cpu().numpy(),one_hot.cpu().numpy())

    recall = recall_score(y_true_res, y_pred_res, average=None)
    precision = precision_score(y_true_res, y_pred_res, average=None)
    f1 = f1_score(y_true_res, y_pred_res, average=None)

    # Calculate macro and weighted average scores
    macro_recall = np.mean(recall)
    macro_precision = np.mean(precision)
    macro_f1 = np.mean(f1)
    weighted_recall = np.average(recall, weights=np.sum(y_true_res, axis=0))
    weighted_precision = np.average(precision, weights=np.sum(y_true_res, axis=0))
    weighted_f1 = np.average(f1, weights=np.sum(y_true_res, axis=0))

    # Create a dictionary with the results
    results = {"recall": recall, "precision": precision, "f1": f1,
               "macro_recall": macro_recall, "macro_precision": macro_precision, "macro_f1": macro_f1,
               "weighted_recall": weighted_recall, "weighted_precision": weighted_precision, "weighted_f1": weighted_f1}
    print()
    print(f'macro_recall: {results["macro_recall"]}  macro_precision: {results["macro_precision"]} macro_F1: {results["macro_f1"]}')
    print(f'weighted_recall: {results["weighted_recall"]}  weighted_precision: {results["weighted_precision"]} weighted_F1: {results["weighted_f1"]}')
    model.train()
    binaryCode_Model.train()
    sourceCode_Model.train()


In [ ]:
train()

6734it [00:01, 4852.77it/s]
100%|██████████| 8201/8201 [00:21<00:00, 386.28it/s]


new: 7980
old: 4641
total_functions: 8104


  0%|          | 20/8104 [00:04<29:03,  4.64it/s]

epoch=0, cur_step=10, loss=1.075961410999298


  0%|          | 40/8104 [00:08<26:43,  5.03it/s]

...

epoch=4, cur_step=20240, loss=0.4002292333382963


100%|█████████▉| 8084/8104 [31:01<00:04,  4.68it/s]

epoch=4, cur_step=20250, loss=0.2819549472381671


100%|██████████| 8104/8104 [31:06<00:00,  4.34it/s]

epoch=4, cur_step=20260, loss=0.3039265433047428
average loss: 0.31532442313141074


In [ ]:
import os
import torch
import torch.nn.functional as F
import binaryBERT
import codeBERT
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from config import *
from transformers import set_seed, AutoTokenizer
from collections import defaultdict
from tqdm import tqdm
import config
import random
import numpy as np
import pickle
import math
import copy
from sklearn.metrics import precision_score, recall_score, f1_score
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA'] = '1'
seed = 628
set_seed(seed)
dataset_path = "/content/source_vs_binary/"
model_path = "/content/drive/MyDrive/Colab/Research/model_state"
sourceCode_Model = codeBERT.MyModel(codeBERT.pretrain_model,codeBERT.config).to(device)
binaryCode_Model = binaryBERT.MyModel().to(device)
binaryCode_Model.load_state_dict(torch.load('model_state/binaryCode_Model1.dat'), strict=False)
sourceCode_Model.load_state_dict(torch.load('model_state/sourceCode_Model1.dat'), strict=False)
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")
binaryDict = binaryBERT.load_tokens()
func_length = 512

Some weights of the model checkpoint at neulab/codebert-c were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-c and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The argument `trust_re

In [ ]:
model = MLP().to(device)
model.load_state_dict(torch.load('model_state/classification_model1.dat'), strict=False)
test2(model)

6734it [00:01, 3905.12it/s]
100%|██████████| 2051/2051 [00:05<00:00, 382.56it/s]


new: 2001
old: 1260
total_functions: 2031


100%|██████████| 2031/2031 [02:35<00:00, 13.03it/s]


macro_recall: 0.6132882948680495  macro_precision: 0.6413872069294534 macro_F1: 0.619825566119438
weighted_recall: 0.7188139059304703  weighted_precision: 0.7144201839067034 weighted_F1: 0.71343965954286
